In [3]:
import numpy as np
import pandas as pd
from my_gmd import *
from scipy.stats import norm
from scipy.optimize import fixed_point
import matplotlib.pyplot as plt
from warnings import warn
from typing import Optional
plt.style.use('bmh')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Analysis

In [2]:
paper_simul = GMD_simluation(tmax=30, 
                        sigma_price=0.5, 
                        proba_jump=0.001, 
                        alpha=0.5, 
                        eta=0.5, 
                        sigma_noise=0.05,
                        V0=100, 
                        multiplier=50)

In [ ]:
s1 = GMD_simluation()